In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


LLM 객체를 정의합니다.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법 1. from_template() 메서드를 사용하여 PromptTemplate 객체 생성
- 치환될 변수 `{ 변수 }`로 묶어서 템플릿을 정의합니다.

In [4]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_teplate 메서드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

`country`변수에 값을 넣어서 문장을 생성할 수 있습니다.

In [5]:
# prompt 생성. format 메서드를 이용하여 변수에 값을 넣어준다.
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [8]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke({"country" : "대한민국"}).content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
추가 유효성 검사를 위해 `input_variables`를 명시적으로 지정하세요. </br>

이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킵니다.

In [9]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [10]:
# prompt 생성
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [11]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2" : "미국" # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [13]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [15]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [16]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [17]:
chain = prompt_partial | llm

In [18]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [19]:
chain.invoke({"country1" : "서울", "country2" : "호주"}).content

'서울은 대한민국의 수도이고, 호주의 수도는 캔버라입니다.'

### `partial_variables` : 부분 변수 채움
`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. </br>

이 사용 사례는 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우입니다. </br>

대표적인 예가 날짜나 시간입니다. </br>

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 날짜를 반환하는 함수를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.

In [20]:
# 다음 코드는 오늘 날짜를 구하는 파이썬 코드입니다.
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'December 27'

In [21]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [26]:
prompt = PromptTemplate(
    template="오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today":get_today # dictionary 형태로 partital_variable를 전달
    },
)

prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x1501c62a0>}, template='오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [23]:
# prompt 생성
prompt.format(n=3)

'오늘 날짜는 December 27입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [24]:
# chain 생성
chain = prompt | llm

In [27]:
# chain을 실행한 후 결과를 확인합니다.
print(chain.invoke(3).content)

1. 박신양 - 1954년 12월 27일
2. 에밀리아 클라크 - 1996년 12월 27일
3. 류준열 - 1986년 12월 27일


In [29]:
# chain을 실행한 후 결과를 확인합니다.
print(chain.invoke({"today" : "January 02", "n" : 3}).content)

1. Kate Bosworth - 1983년 1월 2일
2. Taye Diggs - 1971년 1월 2일
3. Christy Turlington - 1969년 1월 2일


### 파일로부터 template 읽어오기

In [30]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [31]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [36]:
prompt2 = load_prompt("prompts/capital.yaml")
prompt2

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')

In [37]:
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [47]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o", temperature=0) | StrOutputParser()

In [48]:
answer = chain.stream({"country" : "대한민국"})

In [49]:
for token in answer:
    print(token, end="", flush=True)

서울은 대한민국의 수도로, 다양한 특징을 가지고 있습니다.

1. 면적: 서울의 면적은 약 605.21㎢로, 대한민국의 정치, 경제, 문화의 중심지입니다.
2. 인구: 서울의 인구는 약 950만 명으로, 대한민국에서 가장 인구가 많은 도시입니다.
3. 역사적 장소: 경복궁, 창덕궁, 덕수궁 등 조선시대의 궁궐들이 있으며, 한양도성, 종묘 등 유네스코 세계문화유산으로 지정된 장소도 많습니다.
4. 특산품: 서울은 전통과 현대가 조화를 이루는 도시로, 한복, 한지, 전통주 등이 유명하며, 다양한 현대 패션과 기술 제품도 특산품으로 꼽힙니다.

### ChatPromptTemplate
`ChatPromptTemplate`은 대화 목록을 프롬프트로 주입하고자 할 때 사용할 수 있습니다. </br>

메시지는 튜플(tuple) 형식으로 구성하며, (`role`,`message`)로 구성하여 리스트로 생성할 수 있습니다. </br>

role
- `"system"` : 시스템 설정 메시지입니다. 주로 전역설정과 관련된 프롬프트입니다.
- `"human"` : 사용자 입력 메시지입니다.
- `"ai"` : AI의 답변 메시지입니다.

In [1]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [2]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [3]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)

messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

생성한 메시지를 바로 주입하여 결과를 받을 수 있습니다.

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

In [5]:
llm.invoke(messages).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

이번에는 체인을 생성해 보겠습니다.

In [10]:
chain = chat_template | llm

In [12]:
chain.invoke({"name" : "teddy", "user_input" : "당신의 이름은 무엇입니까?"}).content

'제 이름은 teddy입니다. 어떻게 도와드릴까요?'

### MessagePlaceholder
또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder`를 제공합니다. </br>

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

`conversation`대화목록을 나중에 추가하고자 할 때 `MessagesPlaceholder`를 사용할 수 있습니다.

In [14]:
chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다.")
    ]
)

'Human: 안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'

In [20]:
chain = chat_prompt | llm

In [21]:
chain.invoke(
    {
        "word_count" : 5,
        "conversation" : [
            ("human", "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."),
            ("ai", "반가워요! 앞으로 잘 부탁드립니다.")
        ]
    }
).content

'테디 입사, 반가운 인사.'